<a href="https://colab.research.google.com/github/damerei/DS-Unit-2-Sprint-3-Classification-Validation/blob/master/DS_Unit_2_Sprint_Challenge_3_Classification_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 _Lambda School Data Science Unit 2_
 
 # Classification & Validation Sprint Challenge

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

#### For this Sprint Challenge, you'll predict whether a person's income exceeds $50k/yr, based on census data.

You can read more about the Adult Census Income dataset at the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/adult

#### Run this cell to load the data:

In [11]:
!pip install category_encoders

     |████████████████████████████████| 92kB 5.3MB/s 


In [0]:
%matplotlib inline
import warnings
import category_encoders as ce
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings(action='ignore', category=DataConversionWarning)


In [0]:
import pandas as pd

columns = ['age', 
           'workclass', 
           'fnlwgt', 
           'education', 
           'education-num', 
           'marital-status', 
           'occupation', 
           'relationship', 
           'race', 
           'sex', 
           'capital-gain', 
           'capital-loss', 
           'hours-per-week', 
           'native-country', 
           'income']

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', 
                 header=None, names=columns)

df['income'] = df['income'].str.strip()

In [3]:
df.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [13]:
df['income'].value_counts()

<=50K    24720
>50K      7841
Name: income, dtype: int64

## Part 1 — Begin with baselines

Split the data into an **X matrix** (all the features) and **y vector** (the target).

(You _don't_ need to split the data into train and test sets here. You'll be asked to do that at the _end_ of Part 1.)

In [63]:
y = df['income'] 
X = df.drop(columns = 'income')

y.value_counts()

<=50K    24720
>50K      7841
Name: income, dtype: int64

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You can answer this question either with a scikit-learn function or with a pandas function.)

In [73]:
y.mode()[0]

'<=50K'

In [18]:
majority = y.mode()[0]

y_pred = [majority] * len(y)

from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred)

0.7591904425539756

What **ROC AUC score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of ROC AUC.)

In [0]:
#By definition, it must be .5 for a majority class baseline.

In this Sprint Challenge, you will use **"Cross-Validation with Independent Test Set"** for your model validaton method.

First, **split the data into `X_train, X_test, y_train, y_test`**. You can include 80% of the data in the train set, and hold out 20% for the test set.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

## Part 2 — Modeling with Logistic Regression!

- You may do exploratory data analysis and visualization, but it is not required.
- You may **use all the features, or select any features** of your choice, as long as you select at least one numeric feature and one categorical feature.
- **Scale your numeric features**, using any scikit-learn [Scaler](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) of your choice.
- **Encode your categorical features**. You may use any encoding (One-Hot, Ordinal, etc) and any library (category_encoders, scikit-learn, pandas, etc) of your choice.
- You may choose to use a pipeline, but it is not required.
- Use a **Logistic Regression** model.
- Use scikit-learn's [**cross_val_score**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function. For [scoring](https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules), use **accuracy**.
- **Print your model's cross-validation accuracy score.**

In [71]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    StandardScaler(), 
    LogisticRegression(solver='lbfgs', max_iter=1000)
)

scores = cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=10, n_jobs=-1, verbose=10)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.3s finished


In [72]:
print("Cross validation scores:", scores)
print("Mean of cross validation scores:", scores.mean())

Cross validation scores: [0.84650806 0.82808903 0.83921719 0.82840691 0.84644914 0.8406298
 0.83563748 0.83794163 0.83640553 0.84139785]
Mean of cross validation scores: 0.8380682609602317


## Part 3 — Modeling with Tree Ensembles!

Part 3 is the same as Part 2, except this time, use a **Random Forest** or **Gradient Boosting** classifier. You may use scikit-learn, xgboost, or any other library. Then, print your model's cross-validation accuracy score.

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier


from sklearn.model_selection import cross_val_score

pipe = make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestClassifier(n_estimators=100, max_depth=4))

score = cross_val_score(pipe,X_train,y_train,scoring='accuracy',cv=10, n_jobs=-1)
score.mean()


   

0.8447096902423679

In [58]:
pipe2 = make_pipeline(
    ce.OrdinalEncoder(),
    XGBClassifier(n_estimators=20, n_jobs=-1))
score = cross_val_score(pipe2,X_train,y_train,scoring='accuracy',cv=10, n_jobs=-1)
score.mean()

0.8483183127698883

## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

Calculate accuracy

In [37]:
#accuracy = (true positive + true negative)/ number of predictions

accuracy = (36 + 85)/(85 + 58 + 8 + 36)

print(accuracy)

0.6470588235294118


Calculate precision

In [38]:
#precision = (true positive)/(true positives + false positives)

precision = (36)/(36 + 58)

print(precision)

0.3829787234042553


Calculate recall

In [39]:
#recall = (true positives)/(true positives + false negatives)

recall = 36/(36+8)

print(recall)

0.8181818181818182


In [40]:
f1 = 2*((precision*recall)/(precision+recall))

print(f1)

0.5217391304347826


In [41]:
#false positive rate = fpr = false positives/total negatives

fpr = 58/(58 + 85)

print(fpr)

0.40559440559440557


## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Experiment with feature selection, preprocessing, categorical encoding, and hyperparameter optimization, to try improving your cross-validation score.

### Part 3
Which model had the best cross-validation score? Refit this model on the train set and do a final evaluation on the held out test set — what is the test score? 

### Part 4
Calculate F1 score and False Positive Rate. 

## Trying to calculate feature importances

In [0]:
models = [make_pipeline(ce.OrdinalEncoder(), LogisticRegression(solver='lbfgs', max_iter=1000)), 
          
          make_pipeline(ce.OrdinalEncoder(), RandomForestClassifier(max_depth=3, n_estimators=100, n_jobs=-1, 
                                 random_state=42)), 
          make_pipeline(ce.OrdinalEncoder(), RandomForestClassifier(max_depth=None, n_estimators=100, n_jobs=-1, 
                                 random_state=42)), 
          make_pipeline(ce.OrdinalEncoder(), XGBClassifier(max_depth=3, n_estimators=100, n_jobs=-1, 
                        random_state=42))]

In [69]:
for model in models:
  name = model.__class__.__name__
  model.fit(X_train, y_train)
  if name == 'LogisticRegression':
    coefficients = pd.Series(model.coef_[0], X_train.columns)
    coefficients.sort_values().plot.barh(color='grey', title=name)
    plt.show()
  else:
    importances = pd.Series(model.feature_importances_, X_train.columns)
    title = f'{name}, max_depth={model.max_depth}'
    importances.sort_values().plot.barh(color='grey', title=title)
    plt.show()

AttributeError: ignored